In [1]:
import geemap
import ee
import os
import pandas as pd
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
os.chdir(r'E:\Work\QTPofShp')

In [6]:
link_roi = ['users/2431566134liumonarch/QTP',
           'users/2431566134liumonarch/QTP_Rectangle_CustomProj',
           'users/2431566134liumonarch/QHandXZ_Province_Proj']

In [7]:
rois = [ee.FeatureCollection(i) for i in link_roi]

In [8]:
Map.addLayer(rois[1], {}, 'roi1')

In [39]:
tavg_df = pd.DataFrame({'roi':['QTP', 'QTP_Rectangle_CustomProj', 'QHandXZ_Province_Proj']})
prcp_df = pd.DataFrame({'roi':['QTP', 'QTP_Rectangle_CustomProj', 'QHandXZ_Province_Proj']})
tavg_df.set_index('roi', inplace=True)
prcp_df.set_index('roi', inplace=True)
for year in range(2000, 2021):
    image = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterDate(str(year), str(year+1))
    tavg_img = image.select('tmmx').mean().add(image.select('tmmn').mean())
    prcp_img = image.select('pr').sum()
    scale_tavg = tavg_img.projection().nominalScale()
    scale_prcp = prcp_img.projection().nominalScale()
    tavg = [tavg_img.reduceRegion(ee.Reducer.mean(), 
                                roi.geometry(), scale=scale_tavg)\
                                .getNumber('tmmx').getInfo()/2 for roi in rois]
    prcp = [prcp_img.reduceRegion(ee.Reducer.mean(), 
                                 roi.geometry(), scale=scale_prcp)\
                                .getNumber('pr').getInfo() for roi in rois]
    tavg_df.insert(year-2000, str(year), tavg)
    prcp_df.insert(year-2000, str(year), prcp)
    print(f'{year} 统计结束！！！')
tavg_df = tavg_df*0.1
prcp_df.to_csv('result/IDAHO_2000_2020_prcp.csv')
tavg_df.to_csv('result/IDAHO_2000_2020_tavg.csv')

2000 统计结束！！！
2001 统计结束！！！
2002 统计结束！！！
2003 统计结束！！！
2004 统计结束！！！
2005 统计结束！！！
2006 统计结束！！！
2007 统计结束！！！
2008 统计结束！！！
2009 统计结束！！！
2010 统计结束！！！
2011 统计结束！！！
2012 统计结束！！！
2013 统计结束！！！
2014 统计结束！！！
2015 统计结束！！！
2016 统计结束！！！
2017 统计结束！！！
2018 统计结束！！！
2019 统计结束！！！
2020 统计结束！！！


In [44]:
prcp_df = pd.read_csv('result/IDAHO_2000_2020_prcp.csv', index_col=0)
prcp_df = prcp_df*10
prcp_df.to_csv('result/IDAHO_2000_2020_prcp.csv')

In [47]:
tavg_df = pd.DataFrame({'roi':['QTP', 'QTP_Rectangle_CustomProj', 'QHandXZ_Province_Proj']})
prcp_df = pd.DataFrame({'roi':['QTP', 'QTP_Rectangle_CustomProj', 'QHandXZ_Province_Proj']})
tavg_df.set_index('roi', inplace=True)
prcp_df.set_index('roi', inplace=True)
for year in range(2000, 2021):
    image = ee.ImageCollection("NASA/NEX-GDDP")\
                .filterDate(str(year), str(year+1))
    tavg_img = image.select('tasmin').mean().add(image.select('tasmax').mean())
    prcp_img = image.select('pr').sum()
    scale_tavg = tavg_img.projection().nominalScale()
    scale_prcp = prcp_img.projection().nominalScale()
    tavg = [tavg_img.reduceRegion(ee.Reducer.mean(), 
                                roi.geometry(), scale=scale_tavg)\
                                .getNumber('tasmin').getInfo()/2 for roi in rois]
    prcp = [prcp_img.reduceRegion(ee.Reducer.mean(), 
                                 roi.geometry(), scale=scale_prcp)\
                                .getNumber('pr').getInfo() for roi in rois]
    tavg_df.insert(year-2000, str(year), tavg)
    prcp_df.insert(year-2000, str(year), prcp)
    print(f'{year} 统计结束！！！')
prcp_df = prcp_df*1000
tavg_df = tavg_df-273.15
prcp_df.to_csv('result/NASA_2000_2020_prcp.csv')
tavg_df.to_csv('result/NASA_2000_2020_tavg.csv')

2000 统计结束！！！
2001 统计结束！！！
2002 统计结束！！！
2003 统计结束！！！
2004 统计结束！！！
2005 统计结束！！！
2006 统计结束！！！
2007 统计结束！！！


ProxyError: ('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response'))

In [51]:
for year in range(2008, 2021):
    image = ee.ImageCollection("NASA/NEX-GDDP")\
                .filterDate(str(year), str(year+1))
    tavg_img = image.select('tasmin').mean().add(image.select('tasmax').mean())
    prcp_img = image.select('pr').sum()
    scale_tavg = tavg_img.projection().nominalScale()
    scale_prcp = prcp_img.projection().nominalScale()
    tavg = [tavg_img.reduceRegion(ee.Reducer.mean(), 
                                roi.geometry(), scale=scale_tavg)\
                                .getNumber('tasmin').getInfo()/2 for roi in rois]
    prcp = [prcp_img.reduceRegion(ee.Reducer.mean(), 
                                 roi.geometry(), scale=scale_prcp)\
                                .getNumber('pr').getInfo() for roi in rois]
    tavg_df.insert(year-2000, str(year), tavg)
    prcp_df.insert(year-2000, str(year), prcp)
    print(f'{year} 统计结束！！！')

2008 统计结束！！！
2009 统计结束！！！
2010 统计结束！！！
2011 统计结束！！！
2012 统计结束！！！
2013 统计结束！！！
2014 统计结束！！！
2015 统计结束！！！
2016 统计结束！！！
2017 统计结束！！！
2018 统计结束！！！
2019 统计结束！！！
2020 统计结束！！！


In [52]:
prcp_df = prcp_df*1000
tavg_df = tavg_df-273.15
prcp_df.to_csv('result/NASA_2000_2020_prcp.csv')
tavg_df.to_csv('result/NASA_2000_2020_tavg.csv')